In [1]:
!pip install openpyxl
!pip install mlxtend

     |████████████████████████████████| 244 kB 604 kB/s 


In [2]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

In [3]:
raw_data = pd.read_excel('../input/uci-online-retail-ii-data-set/online_retail_II.xlsx',sheet_name='Year 2010-2011')

# Preparing Dataset

In [4]:
def prepare_retail(dataframe):
    # preparing dataset
    dataframe.dropna(inplace=True)
    dataframe = dataframe[~dataframe["Invoice"].str.contains("C", na=False)]
    dataframe = dataframe[dataframe["Quantity"] > 0]
    dataframe = dataframe[dataframe["Price"] > 0]
    return dataframe


In [5]:
df = prepare_retail(raw_data)

# Creating Apriori Datastructure

In [6]:

def create_apriori_datastructure(dataframe, id=False):
    ############################################
    # Preparing ARL Datastructure (Invoice-Product Matrix)
    ############################################

    # We need to create below structure:

    # Rows represents transactions (invoice, shopping cart etc.), columns represents products
    # We simulate as binary that which transaction (invoice, shopping cart etc.) contains which products
    # If the product is in the invoice, the intersection cell will be "1". If is not, it will be "0"

    # Description   Product1   Product2    Product3
    # Invoice
    # Invoice1          0         1          0
    # Invoice2          1         0          1
    # Invoice3          0         0         0
    # Invoice4          1         0         0
    # Invoice5          0         0         1
    if id:
        grouped = germany_df.groupby(
            ['Invoice', 'StockCode'], as_index=False).agg({'Quantity': 'sum'})
        apriori_datastructure = pd.pivot(data=grouped, index='Invoice', columns='StockCode', values='Quantity').fillna(
            0).applymap(lambda x: 1 if x > 0 else 0)
        return apriori_datastructure
    else:
        grouped = germany_df.groupby(
            ['Invoice', 'Description'], as_index=False).agg({'Quantity': 'sum'})
        apriori_datastructure = pd.pivot(data=grouped, index='Invoice', columns='Description', values='Quantity').fillna(
            0).applymap(lambda x: 1 if x > 0 else 0)
        return apriori_datastructure

# Selecting Germany Based Invoices

In [7]:
germany_df = df[df['Country'] == 'Germany'] # In this example, we will cover just Germany based transactions

germany_df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
1109,536527,22809,SET OF 6 T-LIGHTS SANTA,6,2010-12-01 13:04:00,2.95,12662.0,Germany
1110,536527,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,6,2010-12-01 13:04:00,2.55,12662.0,Germany
1111,536527,84945,MULTI COLOUR SILVER T-LIGHT HOLDER,12,2010-12-01 13:04:00,0.85,12662.0,Germany
1112,536527,22242,5 HOOK HANGER MAGIC TOADSTOOL,12,2010-12-01 13:04:00,1.65,12662.0,Germany
1113,536527,22244,3 HOOK HANGER MAGIC GARDEN,12,2010-12-01 13:04:00,1.95,12662.0,Germany


In [8]:
germany_apriori_df = create_apriori_datastructure(germany_df,True)

In [9]:
germany_apriori_df.head() # we see the Invoice-Product matrix

StockCode,10002,10125,10135,11001,15034,15036,15039,16008,16011,16014,...,90161B,90161C,90161D,90201A,90201B,90201C,90201D,90202D,M,POST
Invoice,,,,,,,,,,,,,,,,,,,,,
536527,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
536840,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
536861,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
536967,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
536983,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


# Learning Rules (ARL)

In [10]:

def get_rules(apriori_df, min_support=0.01):
    # Possibilities of all possible product combinations
    # We say that the products that can be sold together with a min 0.01 probability should come. the probability that each product will be sold together with each other. Applying apriori algorithm.
    frequent_itemsets = apriori(
        apriori_df, min_support=min_support, use_colnames=True)
    # Extracting Association Rules
    # We extract association rules by using the support metric from the dataset that we applied the apriori algorithm.
    rules = association_rules(
        frequent_itemsets, metric="support", min_threshold=min_support)
    # antecedents -> the first product(s)
    # consequents -> the next product(s)
    # antecedent support -> probability of observing the first product(s)
    # consequent support -> probability of observing the next product(s)
    # support -> probability of observing the next product(s) and the first product(s) together
    # confidence -> probability of observing the next product(s) when sold the first product(s)
    # lift -> When the first product is sold, the probability of selling the next product(s) increases by a factor of lift.
    # leverage -> Similar to lift, but tends to prioritize the higher support values.
    # conviction -> probability of observing the antecedents without consequents
    return rules

In [11]:
germany_rules = get_rules(germany_apriori_df)

In [12]:
germany_rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(POST),(10125),0.818381,0.013129,0.013129,0.016043,1.221925,0.002384,1.002961
1,(10125),(POST),0.013129,0.818381,0.013129,1.000000,1.221925,0.002384,inf
2,(15036),(POST),0.019694,0.818381,0.015317,0.777778,0.950386,-0.000800,0.817287
3,(POST),(15036),0.818381,0.019694,0.015317,0.018717,0.950386,-0.000800,0.999004
4,(16016),(POST),0.010941,0.818381,0.010941,1.000000,1.221925,0.001987,inf


# Creating Some Utility Functions

In [13]:
def get_item_name(dataframe, stock_code):
    # To find out the product name of the id in the data structure
    # if stock_code is a list, it will return the list by created of item names
    if type(stock_code) != list:
        product_name = dataframe[dataframe["StockCode"] ==
                                 stock_code][["Description"]].values[0].tolist()
        return product_name
    else:
        product_names = [dataframe[dataframe["StockCode"] == product][[
            "Description"]].values[0].tolist()[0] for product in stock_code]
        return product_names

In [14]:
get_item_name(germany_df,10125)

['MINI FUNKY DESIGN TAPES']

In [15]:
def get_golden_shot(target_id,dataframe,rules):
    target_product = get_item_name(dataframe,target_id)[0]
    recomended_product_ids = recommend_products(rules, target_id)
    recomended_product_names = get_item_name(dataframe,recommend_products(rules, target_id))
    print(f'Target Product ID (which is in the cart): {target_id}\nProduct Name: {target_product}')
    print(f'Recommended Products: {recomended_product_ids}\nProduct Names: {recomended_product_names}')

# Recommending Products

In [16]:
def recommend_products(rules_df, product_id, rec_count=5):
    # rules_df -> the dataframe that we extracted rules
    # product_id -> the product id which is in the cart
    # rec_count -> count of recommended products
    sorted_rules = rules_df.sort_values('lift', ascending=False) # we are sorting the rules dataframe by using "lift" metric
    recommended_products = []  # creating an empty list for holding the recommended products

    for i, product in sorted_rules["antecedents"].items(): # loop on the first products (the products which are in the cart)
        for j in list(product):  # assign to a list for each product
            if j == product_id:  # if the list you return is equal to product_id, which means the product id in the cart
                # consequences column's first product id add to recommended products list 
                recommended_products.append(
                    list(sorted_rules.iloc[i]["consequents"]))
                
    recommended_products = list({item for item_list in recommended_products for item in item_list}) # get unique products

    return recommended_products[:rec_count] # return the recommended_products list by using rec_count limiter

# Testing Recommendation

In [17]:
# simulating some products like they are in cart
TARGET_PRODUCT_ID_1 = 21987
TARGET_PRODUCT_ID_2 = 23235
TARGET_PRODUCT_ID_3 = 22747

In [18]:
get_item_name(germany_df, [TARGET_PRODUCT_ID_1,TARGET_PRODUCT_ID_2, TARGET_PRODUCT_ID_3]) # what are their names?

['PACK OF 6 SKULL PAPER CUPS',
 'STORAGE TIN VINTAGE LEAF',
 "POPPY'S PLAYHOUSE BATHROOM"]

In [19]:
get_golden_shot(TARGET_PRODUCT_ID_1,germany_df,germany_rules)

Target Product ID (which is in the cart): 21987
Product Name: PACK OF 6 SKULL PAPER CUPS
Recommended Products: [21124, 22029, 22037, 22423, 22554]
Product Names: ['SET/10 BLUE POLKADOT PARTY CANDLES', 'SPACEBOY BIRTHDAY CARD', 'ROBOT BIRTHDAY CARD', 'REGENCY CAKESTAND 3 TIER', 'PLASTERS IN TIN WOODLAND ANIMALS']


In [20]:
get_golden_shot(TARGET_PRODUCT_ID_2,germany_df,germany_rules)

Target Product ID (which is in the cart): 23235
Product Name: STORAGE TIN VINTAGE LEAF
Recommended Products: [23298, 21244, 20750, 23311, 22423]
Product Names: ['SPOTTY BUNTING', 'BLUE POLKADOT PLATE ', 'RED RETROSPOT MINI CASES', 'VINTAGE CHRISTMAS STOCKING ', 'REGENCY CAKESTAND 3 TIER']


In [21]:
get_golden_shot(TARGET_PRODUCT_ID_3,germany_df,germany_rules)

Target Product ID (which is in the cart): 22747
Product Name: POPPY'S PLAYHOUSE BATHROOM
Recommended Products: ['85049E', 20750, 22423, 22554, 22555]
Product Names: ['SCANDINAVIAN REDS RIBBONS', 'RED RETROSPOT MINI CASES', 'REGENCY CAKESTAND 3 TIER', 'PLASTERS IN TIN WOODLAND ANIMALS', 'PLASTERS IN TIN STRONGMAN']
